## Imports

In [11]:
import json
from shapely.geometry import shape, Point
import geopy.distance
import pprint

### Check if point inside polygon

In [30]:
# load GeoJSON file containing sectors
# with open("Datos Abiertos - Bogota/Ordenamiento_Territorial - Sector_Catastral/Raw-data-sector-catastral-april-2023.geojson") as f:
#     js = json.load(f)

# def check_point_inside_polygon(point=Point(-74.066837, 4.732316), features=js["features"]): 
def check_point_inside_polygon(point, features): 
    for feature in features:
        polygon = shape(feature["geometry"])
        if polygon.contains(point):
            return feature["properties"]

### Check points in radius

In [32]:
def check_point_within_radius(center, geometry, radius=1000):
    '''
        center = (lat, long)
        geometry = {
        "type": "Point", 
        "coordinates": (0.0, 0.0)
        }
    '''
    point = shape(geometry)
    if point.geom_type == 'Point':
        pointCoord = (list(point.coords)[0][1], list(point.coords)[0][0])
        if geopy.distance.geodesic(center, pointCoord).m <= radius:
            return True
    return False

### Get distance between points

In [31]:
def get_distance_between_points(center, geometry):
    '''
        center = (lat, long)
        geometry = {
        "type": "Point", 
        "coordinates": (0.0, 0.0)
        }
    '''
    point = shape(geometry)
    if point.geom_type == 'Point':
        pointCoord = (list(point.coords)[0][1], list(point.coords)[0][0])
        return geopy.distance.geodesic(center, pointCoord).m
    return None

### Get polygons in radius

In [21]:
# Raw data should be processed with QGIS before (to WGS84) in order to have Type and Coordinates inside geometry
# with open("Datos Abiertos - Bogota/Ordenamiento_Territorial - Parques POT/data-parques-feb-2023-processed.geojson") as f:
#     js = json.load(f)

# def get_polygons_in_radius(center=Point(-74.066837, 4.732316), features=js["features"], radius=1000):
def get_polygons_in_radius(center, features, radius):
    results = []
    for feature in features:
        poly = shape(feature["geometry"])
        feature["dist2poly1"] = poly.distance(center) * 111194.93  # Degress to M considering earth
        if feature["dist2poly1"] <= radius:
            feature.pop("geometry", None)
            results.append(feature)
    return sorted(results, key=lambda d: d["dist2poly1"])